# Linear Regression for Salary Data

1. Dataset Information
2. Data Preprocessing
3. Train Models

| | Model |
|:--| :--|
| A | Job Title feature modified using One Hot Encoding |
| B | Job Title feature modified using Label Encoding |
| C | Job Title feature dropped from dataset |

4. Comparison of Models

In [19]:
## imports
import pandas as pd 
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split



## 1. Dataset Information

In [2]:
data = pd.read_csv('Salary Data.csv')
print(data.head(5))
print(data.info())

    Age  Gender Education Level          Job Title  Years of Experience   
0  32.0    Male      Bachelor's  Software Engineer                  5.0  \
1  28.0  Female        Master's       Data Analyst                  3.0   
2  45.0    Male             PhD     Senior Manager                 15.0   
3  36.0  Female      Bachelor's    Sales Associate                  7.0   
4  52.0    Male        Master's           Director                 20.0   

     Salary  
0   90000.0  
1   65000.0  
2  150000.0  
3   60000.0  
4  200000.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  373 non-null    float64
 1   Gender               373 non-null    object 
 2   Education Level      373 non-null    object 
 3   Job Title            373 non-null    object 
 4   Years of Experience  373 non-null    float64
 5   Salary

In [3]:
salary_dist = px.histogram(data, x="Salary")
salary_dist.show()

In [4]:
gender_count = data['Gender'].value_counts().reset_index()
gender_dist = px.pie(gender_count, values='count', names="Gender", title="Count by Gender")
gender_dist.show()

In [5]:
edu_count = data['Education Level'].value_counts().reset_index()
edu_dist = px.pie(edu_count, values='count', names="Education Level", title="Education Level by Highest Degree")
edu_dist.show()

## 2. Data Preprocessing

https://scikit-learn.org/stable/modules/preprocessing.html
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder

In [6]:
####################
#  Clean data
####################
# Remove na
clean_df = data.dropna()
# Remove duplicates
clean_df = clean_df.drop_duplicates()
# Check for negative salary
negative_salaries = clean_df[clean_df['Salary'] < 0]
if (negative_salaries.shape[0] > 0):
    print("Warning: There are salaries < 0!!")

####################
#  Converting the Categorial data into numbers
####################
label_encoder = LabelEncoder()
# Education Level
clean_df['Education Level'] = label_encoder.fit_transform(clean_df['Education Level'])
# One Hot Encode Gender & Job Title
job_1hot_df = pd.get_dummies(clean_df)

job_encoder = LabelEncoder()
job_label_df  = clean_df
job_label_df['Job Title'] = label_encoder.fit_transform(clean_df['Job Title'])
job_label_df = pd.get_dummies(job_label_df)

no_job_df = clean_df.drop(['Job Title'], axis=1)
no_job_df = pd.get_dummies(no_job_df)

print("One Hot")
print(job_1hot_df.head(5))
print("Label Encoding")
print(job_label_df.head(5))
print("No Job")
print(no_job_df.head(5))

One Hot
    Age  Education Level  Years of Experience    Salary  Gender_Female   
0  32.0                0                  5.0   90000.0          False  \
1  28.0                1                  3.0   65000.0           True   
2  45.0                2                 15.0  150000.0          False   
3  36.0                0                  7.0   60000.0           True   
4  52.0                1                 20.0  200000.0          False   

   Gender_Male  Job Title_Account Manager  Job Title_Accountant   
0         True                      False                 False  \
1        False                      False                 False   
2         True                      False                 False   
3        False                      False                 False   
4         True                      False                 False   

   Job Title_Administrative Assistant  Job Title_Business Analyst  ...   
0                               False                       False  ...

## 3. Train Models

In [25]:
all_mse = []
all_r2 = []
all_rmse = []
label = ["1-Hot", "Label Encoder", "Drop Job Title"]

#### Model A - Job Title using One Hot Encoder

In [26]:
x = job_1hot_df.drop(['Salary'], axis=1)
y = job_1hot_df['Salary']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= .2, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, train_size=.5, random_state=42)

# print(f'X Train: {len(x_train)}, Y Train: {len(y_train)}')
# print(f'X Test: {len(x_test)}, Y Test: {len(y_test)}')
# print(f'X Valid: {len(y_valid)}, Y Valid: {len(y_valid)}')

## Create model to estimate salary based all columns

model = LinearRegression()

model.fit(x_train,y_train)

y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
# acc = accuracy_score(y_true=y_test, y_pred=y_pred)

# print(f'Accuracy: {acc}')
print(f'Mean Squared Error: {mse}')
all_mse.append(mse)
all_rmse.append(rmse)
all_r2.append(r2)
# all_acc.append(acc)

# line equation
coeff = model.coef_
intercept = model.intercept_

coeff_df = pd.DataFrame({
    'Feature': x.columns.tolist(),
    'Coefficients': coeff,
    '|Coefficients|': [abs(i) for i in coeff]
})

coeff_df.sort_values(by='|Coefficients|', ascending=False).head(10)

Mean Squared Error: 1.3421021865406464e+30


,Feature,Coefficients,|Coefficients|
3,Gender_Female,1.706335e+16,1.706335e+16
4,Gender_Male,1.706335e+16,1.706335e+16
20,Job Title_Customer Success Manager,-1.423449e+16,1.423449e+16
5,Job Title_Account Manager,9.665781e+15,9.665781e+15
15,Job Title_Copywriter,6.827317e+15,6.827317e+15
11,Job Title_CEO,1.991679e+14,1.991679e+14
13,Job Title_Chief Technology Officer,1.991679e+14,1.991679e+14
12,Job Title_Chief Data Officer,1.991679e+14,1.991679e+14
176,Job Title_VP of Finance,1.991679e+14,1.991679e+14
104,Job Title_Research Director,1.991679e+14,1.991679e+14


#### Model B -  Job Title using Label Encoding

In [27]:
x = job_label_df.drop(['Salary'], axis=1)
y = job_label_df['Salary']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= .2, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, train_size=.5, random_state=42)

# print(f'X Train: {len(x_train)}, Y Train: {len(y_train)}')
# print(f'X Test: {len(x_test)}, Y Test: {len(y_test)}')
# print(f'X Valid: {len(y_valid)}, Y Valid: {len(y_valid)}')

## Create model to estimate salary based all columns
model = LinearRegression()

model.fit(x_train,y_train)

y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
all_mse.append(mse)
all_rmse.append(rmse)
all_r2.append(r2)

# line equation
coeff = model.coef_
intercept = model.intercept_

coeff_df = pd.DataFrame({
    'Feature': x.columns.tolist(),
    'Coefficients': coeff,
    '|Coefficients|': [abs(i) for i in coeff]
})

coeff_df.sort_values(by='|Coefficients|', ascending=False).head(40)

Mean Squared Error: 236005834.4863942


,Feature,Coefficients,|Coefficients|
1,Education Level,15422.735860,15422.735860
4,Gender_Female,-3694.539168,3694.539168
5,Gender_Male,3694.539168,3694.539168
3,Years of Experience,2892.128506,2892.128506
0,Age,2812.906962,2812.906962
2,Job Title,19.576956,19.576956


#### Mobel C - Job Title Removed

In [28]:
x = no_job_df.drop(['Salary'], axis=1)
y = no_job_df['Salary']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= .2, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, train_size=.5, random_state=42)

# print(f'X Train: {len(x_train)}, Y Train: {len(y_train)}')
# print(f'X Test: {len(x_test)}, Y Test: {len(y_test)}')
# print(f'X Valid: {len(y_valid)}, Y Valid: {len(y_valid)}')

## Create model to estimate salary based all columns
model = LinearRegression()

model.fit(x_train,y_train)

y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
all_mse.append(mse)
all_rmse.append(rmse)
all_r2.append(r2)

# line equation
coeff = model.coef_
intercept = model.intercept_

coeff_df = pd.DataFrame({
    'Feature': x.columns.tolist(),
    'Coefficients': coeff,
    '|Coefficients|': [abs(i) for i in coeff]
})

coeff_df.sort_values(by='|Coefficients|', ascending=False).head(40)

Mean Squared Error: 231440652.4771454


,Feature,Coefficients,|Coefficients|
1,Education Level,15488.204557,15488.204557
4,Gender_Male,3750.154843,3750.154843
3,Gender_Female,-3750.154843,3750.154843
0,Age,2886.497521,2886.497521
2,Years of Experience,2822.257560,2822.257560


## 4. Comparison of Models

In [30]:
compare_df = pd.DataFrame({
    'Model': label,
    'MSE': all_mse, 
    'RMSE': all_rmse,
    'R2': all_r2
})

print(compare_df)

            Model           MSE          RMSE            R2
0           1-Hot  1.342102e+30  1.158491e+15 -8.694781e+20
1   Label Encoder  2.360058e+08  1.536248e+04  8.471041e-01
2  Drop Job Title  2.314407e+08  1.521317e+04  8.500616e-01
